In [ ]:
# https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

In [76]:
import pandas as pd
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [93]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = replace_contractions(text)
    return text

def remove_nonascii(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_url(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    return text

def remove_digit_from_string(text):
    text = re.sub("\S*\d\S*", "", text).strip()
    return text

def remove_digit(text):
    text = re.sub(r"\b\d+\b", " ", text)
    return text

def remove_punctuations(text):
    text = re.sub(r'[^\w]|_',' ',text)
    return text

def remove_byte_str(text):
    text = text.replace("b'",'')
    text = text.replace('b"','')
    return text

def remove_additional_white_spaces(text):
    text = re.sub('[\s]+', ' ', text)
    return text

def remove_retweet_mention(text):
    text = re.sub('RT @[\w_]+: ', '', text)
    return text

def remove_single_letter(text):
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    return text
    
def do_preprocessing(text):
    text = denoise_text(text)
    text = remove_byte_str(text)
    text = remove_retweet_mention(text)
    text = remove_nonascii(text)
    text = remove_single_letter(text)
    text = remove_url(text)
    text = remove_digit_from_string(text)
    text = remove_digit(text)
    text = remove_punctuations(text)
    text = remove_additional_white_spaces(text)
    text = text.lower()

    return text

In [87]:
#test code
text = ['\n\nKegiatan yang sama berlangsung di 20 <head>tes </head> puskesmas di kabupaten </br> itu. 4L4y https://t.co/dqVoKKu5qI','Adi Utarini masuk daftar "Nature\xe2\x80\x99s 10: Ten People Who Helped Shape Science in 2020" dari jurnal sains Nature. Sementara Tri Mumpuni termasuk 22 Most Influential Muslim Scientists. https://t.co/9fkpB8B1gC',' \n\nBandara dengan landas pacu 900x30 meter dan terminal berkapasitas hingga 36.000 penumpang per tahun itu adalah wujud kehadiran pemerintah membangun wilayah tertinggal, terpencil, dan terluar','Vaksinasi massal untuk pekerja publik dan kelompok lansia sedang digelar di Puskesmas Getengan, Kabupaten Tana Toraja, ketika saya datang berkunjung pagi tadi. \n\nKegiatan yang sama berlangsung di 20 puskesmas di kabupaten itu. https://t.co/dqVoKKu5qI']

for n in text:
    print(do_preprocessing(n))

kegiatan yang sama berlangsung di tes puskesmas di kabupaten itu 
adi utarini masuk daftar natureas ten people who helped shape science in dari jurnal sains nature sementara tri mumpuni termasuk most influential muslim scientists 
bandara dengan landas pacu meter dan terminal berkapasitas hingga penumpang per tahun itu adalah wujud kehadiran pemerintah membangun wilayah tertinggal terpencil dan terluar
vaksinasi massal untuk pekerja publik dan kelompok lansia sedang digelar di puskesmas getengan kabupaten tana toraja ketika saya datang berkunjung pagi tadi kegiatan yang sama berlangsung di puskesmas di kabupaten itu 


In [94]:
import pandas as pd

df = pd.read_excel('../all_tweets/all_data.xlsx', sheet_name='Sheet2')

#remove newline from texts
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

for t in df['text']:
    print('Ori:\n',t)
    print('New:\n',do_preprocessing(t))
    print('\n')

Ori:
 b"Vaksinasi massal untuk pekerja publik dan kelompok lansia sedang digelar di Puskesmas Ge'tengan, Kabupaten Tana Toraja, ketika saya datang berkunjung pagi tadi. Kegiatan yang sama berlangsung di 20 puskesmas di kabupaten itu. https://t.co/dqVoKKu5qI"
New:
 vaksinasi massal untuk pekerja publik dan kelompok lansia sedang digelar di puskesmas ge tengan kabupaten tana toraja ketika saya datang berkunjung pagi tadi kegiatan yang sama berlangsung di puskesmas di kabupaten itu 


Ori:
 b'Dari Toraja, saya juga meresmikan secara virtual Bandara Pantar di Kab. Alor, NTT. Bandara dengan landas pacu 900x30 meter dan terminal berkapasitas hingga 36.000 penumpang per tahun itu adalah wujud kehadiran pemerintah membangun wilayah tertinggal, terpencil, dan terluar.'
New:
 dari toraja saya juga meresmikan secara virtual bandara pantar di kab alor ntt bandara dengan landas pacu meter dan terminal berkapasitas hingga penumpang per tahun itu adalah wujud kehadiran pemerintah membangun wilayah te